In [ ]:
#This code should be ran from google colab
from google.colab import drive  # type: ignore
drive.mount('/content/drive')

<h4>Let's import the required packages<h4>

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

<h4>Load the pandas dataframe</h4>

In [ ]:
df_train= pd.read_csv("drive/MyDrive/cassava-leaf-disease-classification/train.csv")
df_train['label']= df_train['label'].astype(str)
df_train.head()

In [ ]:
import os;
print(os.getcwd())


<h2><b>Data Visualization</b></h2>
<h4>Diseases and their classes</h4>

In [ ]:
#data Visualization
import matplotlib.pyplot as plt

# Creating data
year = df_train['DiseaseName']
production = df_train['label']

# Plotting barchart
plt.bar(year, production)

# Saving the figure.
plt.savefig("output.jpg")

# Saving figure by changing parameter values
plt.savefig("output1", facecolor='y', bbox_inches="tight",
			pad_inches=0.3, transparent=True)


<h4>Number of images in each class</h4>

In [ ]:
ax = df_train['DiseaseName'].value_counts().plot(kind='bar',figsize=(14,8), title="Dataset Disease Population by Class")
ax.set_xlabel("Disease Types")
ax.set_ylabel("Num of Images")
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
country_data = df_train["label"]
medal_data = df_train.groupby(df_train["DiseaseName"])['label'].count()
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#8c564b"]
explode = (0, 0, 0, 0.09, 0)  
plt.pie(medal_data, labels=medal_data.index, colors=colors, explode =explode, autopct = '%1.1f%%', startangle=15)
plt.title("Pie Chart of Diseases\n"+"and their frequency")
plt.show()

In [ ]:
medal_data = df_train.groupby(df_train["DiseaseName"])['label'].count()

medal_data

<h2><b>Initialize keras' ImageDataGenerator class</b></h2>

In [ ]:
src_path_train= "drive/MyDrive/cassava-leaf-disease-classification/train_images"
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

#validation_datagen2 = ImageDataGenerator(validation_split= 0.20)

<h2><b>Initialize the training and validation generator</b></h2>

In [ ]:
batch_size= 32
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=src_path_train,
    x_col="image_id",
    y_col="label",
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode="sparse",
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=src_path_train,
    x_col="image_id",
    y_col="label",
    target_size=(128,128),
    batch_size=batch_size,
    class_mode="sparse",
    subset='validation',
    shuffle=True,
    seed=42
)


<h2><b>Defining the CNN</b></h2>

In [ ]:
activation = 'sigmoid'
model = Sequential()
model.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(256, 3, padding='Same', activation=activation, kernel_initializer= 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(256, 3, padding='Same', activation=activation, kernel_initializer= 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))


model.add(Conv2D(512, 4, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(128, activation = activation, kernel_initializer = 'he_uniform'))
model.add(Dense(5, activation = 'softmax'))

model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
print(model.summary()) 

<h2><b>Training our model using fit_generator</b></h2>

In [ ]:
history= model.fit_generator(train_generator,
                    validation_data= validation_generator,
                    steps_per_epoch= len(df_train) * 0.8/batch_size,
                    validation_steps= len(df_train) * 0.2/batch_size,
                    epochs= 20)

<h2><b>Save model</b></h2>

In [ ]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

<h2><b>Plotting the accuracy and loss graphs for the training</b></h2>

In [ ]:
#Plot the Graph
from matplotlib import pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(loss,'r',linewidth=3.0)
plt.plot(val_loss,'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
  
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(accuracy,'r',linewidth=3.0)
plt.plot(val_accuracy,'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

<h2><b>An attempt to improve on the initial CNN model using the EfficientNetB4 transfer learning model</b></h2>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import EfficientNetB4  # type: ignore
from keras.optimizers import Adam
from tensorflow.keras import models, layers  # type: ignore


model = models.Sequential()
model.add(EfficientNetB4(include_top = False, weights = 'imagenet',
                             input_shape = (128, 128, 3)))
    
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(5, activation = "softmax"))

model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])

model.summary()

<h4>Intermediate steps</h4>

In [ ]:
import datetime
import tensorflow as tf
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_save = ModelCheckpoint('./best_baseline_model.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)



<h4>The training</h4>

In [ ]:

history= model.fit_generator(train_generator,
                    validation_data= validation_generator,
                    steps_per_epoch= len(df_train) * 0.8/batch_size,
                    validation_steps= len(df_train) * 0.2/batch_size,
                    epochs= 20,
                    callbacks = [tensorboard_callback,model_save, early_stop, reduce_lr])

In [ ]:
#Plot the Graph
from matplotlib import pyplot as plt
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(loss,'r',linewidth=3.0)
plt.plot(val_loss,'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
  
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(accuracy,'r',linewidth=3.0)
plt.plot(val_accuracy,'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

<h2><b>Using the VGG16 Transfer learning model</b2></h2>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator    # type: ignore
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import VGG16   # type: ignore
from keras.optimizers import Adam
from tensorflow.keras import models, layers # type: ignore 


model1 = models.Sequential()
model1.add(VGG16(include_top = False, weights = 'imagenet',
                             input_shape = (128, 128, 3)))
    
model1.add(layers.GlobalAveragePooling2D())
model1.add(layers.Dense(5, activation = "softmax"))

model1.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])

model1.summary()

<h4>Intermediate steps</h4>

In [ ]:
import datetime
import tensorflow as tf
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model1_save = ModelCheckpoint('./best_baseline_model1.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop1 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr1 = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

<h4>The training and plots</h4>

In [ ]:
history= model1.fit_generator(train_generator,
                    validation_data= validation_generator,
                    steps_per_epoch= len(df_train) * 0.8/batch_size,
                    validation_steps= len(df_train) * 0.2/batch_size,
                    epochs= 20,
                    callbacks = [tensorboard_callback1,model1_save, early_stop1, reduce_lr1])

#Plot the Graph
from matplotlib import pyplot as plt
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(loss,'r',linewidth=3.0)
plt.plot(val_loss,'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
  
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(accuracy,'r',linewidth=3.0)
plt.plot(val_accuracy,'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

<h2><b>Using the inception V3 Transfer learning model</b></h>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import InceptionV3    # type: ignore
from keras.optimizers import Adam
from tensorflow.keras import models, layers  # type: ignore


model3 = models.Sequential()
model3.add(InceptionV3(include_top = False, weights = 'imagenet',
                             input_shape = (128, 128, 3)))
    
model3.add(layers.GlobalAveragePooling2D())
model3.add(layers.Dense(5, activation = "softmax"))

model3.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])

model3.summary()

<h4>Intermediate steps</h4>

In [ ]:
import datetime
import tensorflow as tf
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback3 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model3_save = ModelCheckpoint('./best_baseline_model2.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop3 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr3 = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

<h4>The training and plots</h4>

In [ ]:
history= model3.fit_generator(train_generator,
                    validation_data= validation_generator,
                    steps_per_epoch= len(df_train) * 0.8/batch_size,
                    validation_steps= len(df_train) * 0.2/batch_size,
                    epochs= 20,
                    callbacks = [tensorboard_callback3,model3_save, early_stop3, reduce_lr3])

#Plot the Graph
from matplotlib import pyplot as plt
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(loss,'r',linewidth=3.0)
plt.plot(val_loss,'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
  
# Accuracy Curves


plt.figure(figsize=[8,6])
plt.plot(accuracy,'r',linewidth=3.0)
plt.plot(val_accuracy,'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

<h4>The below code saves the images in the first dataset to different folders/classes</h4>

In [ ]:
dataset_dir = "drive/MyDrive/cassava-leaf-disease-classification"
train_dir= dataset_dir + "/train_images/"

CBB = "/train_imgs_folder/CBB"
CBSD= "/train_imgs_folder/CBSD"
CGM= "/train_imgs_folder/CGM"
CMD= "/train_imgs_folder/CMD"
H=  "/train_imgs_folder/H"

In [ ]:
# This code should only be called once to save the images to the five folders inside train_imgs_folder and that is after you have manually 
# created the train_imgs_folder and the five folders inside it, in your working drive in googlen drive
import cv2
for index, row in df_train.iterrows():
  label= row['label']
  if label== '0':
    #save to 0/CBB folder
    image_name= row['image_id']
    path= train_dir + image_name
    image= cv2.imread(path)
    #cv2.imwrite(dataset_dir + CBB + '/' + image_name, image)

  elif label == '1':
    #save to 1/CBSD folder
    image_name= row['image_id']
    path= train_dir + image_name
    image= cv2.imread(path)
    cv2.imwrite(dataset_dir + CBSD + '/' + image_name, image)

  elif label == '2':
    #save to 2/CGM folder
    image_name= row['image_id']
    path= train_dir + image_name
    image= cv2.imread(path)
    cv2.imwrite(dataset_dir + CGM + '/' + image_name, image)

  elif label == '3':
    #savre to 3/CMD folder
    image_name= row['image_id']
    path= train_dir + image_name
    image= cv2.imread(path)
    cv2.imwrite(dataset_dir + CMD + '/' + image_name, image)

  elif label == '4':
    # save to H/4 folder
    image_name= row['image_id']
    path= train_dir + image_name
    image= cv2.imread(path)
    cv2.imwrite(dataset_dir + H + '/' + image_name, image)


<h4>The code for this section was inspired by codebasics code from this link https://github.com/codebasics/potato-disease-classification/blob/main/training/potato-disease-classification-model.ipynb</h4>

<h2><b>Using the ResNet50 transfer learning model</b></h2>

<h4>Import data into tensorflow dataset object</h4>

In [ ]:
directory= "drive/MyDrive/cassava-leaf-disease-classification/train_images"
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    seed=123,
    shuffle=True,
    image_size=(128, 128),
    batch_size=32
)

In [ ]:
class_names = dataset.class_names
class_names

<h4>Function to Split Dataset</h4>

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

Cache, Shuffle, and Prefetch the Dataset

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import ResNet50 # type: ignore
from keras.optimizers import Adam
from tensorflow.keras import models, layers # type: ignore


model2 = models.Sequential()
model2.add(ResNet50(include_top = False, weights = 'imagenet',
                             input_shape = (128, 128, 3)))
    
model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.Dense(5, activation = "softmax"))

model2.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])

model2.summary()

In [ ]:
import datetime
import tensorflow as tf
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model2_save = ModelCheckpoint('./best_baseline_model2.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop2 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr2 = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

the training and plots

In [ ]:
history= model2.fit(train_ds,
                    validation_data= val_ds,
                    batch_size=32,
                    #steps_per_epoch= len(df_train) * 0.8/BATCH_SIZE,
                    #validation_steps= len(df_train) * 0.2/BATCH_SIZE,
                    verbose=1,
                    epochs= 30,
                    callbacks = [tensorboard_callback2, model2_save, early_stop2, reduce_lr2])


# Save the entire model as a SavedModel.
#!mkdir -p saved_model
model2.save("drive/MyDrive/cassava-leaf-disease-classification/saved_model/my_model")

In [ ]:
#Plot the Graph
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(loss,'r',linewidth=3.0)
plt.plot(val_loss,'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
  
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(accuracy,'r',linewidth=3.0)
plt.plot(val_accuracy,'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)


In [ ]:
#Evaluate model on test_ds
scores = model2.evaluate(test_ds)

Run prediction on a sample image

In [ ]:

import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

Write a function for inference

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img.numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence


Now run inference on few sample images

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i])
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

<h3><b>Using the ResNet50 transfer learning model with weight regularization to prevent overfitting</b></h3>

Import all the Dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

Set all the Constants

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 128
CHANNELS=5
EPOCHS=30

Import data into tensorflow dataset object

In [ ]:
directory= "drive/MyDrive/cassava-leaf-disease-classification/train_imgs_folder"
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    seed=123,
    shuffle=True,
    image_size=(128, 128),
    batch_size=32
)

In [ ]:

class_names = dataset.class_names
class_names

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

Visualize some of the images from our dataset

In [ ]:
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

Function to Split Dataset

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

Cache, Shuffle, and Prefetch the Dataset

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Function to add weight regularization to the Transfer learning model

This code for this function was gotten from this source: @article{ silva2019kerasregularization, title={How to Add Regularization to Keras Pre-trained Models the Right Way}, author={Silva, Thalles Santos}, journal={https://sthalles.github.io}, year={2019} url={https://sthalles.github.io/keras-regularizer/}

In [ ]:
import os
import tempfile

def add_regularization(model, regularizer=tf.keras.regularizers.l2(0.0001)):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
      print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
      return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
              setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import ResNet50
from keras.optimizers import Adam
from tensorflow.keras import models, layers


model6 = models.Sequential()
model6.add(ResNet50(include_top = False, weights = 'imagenet',
                             input_shape = (128, 128, 3)))
#Add regularization using the function below
add_regularization(model6, regularizer=tf.keras.regularizers.l2(0.0001))
    
model6.add(layers.GlobalAveragePooling2D())
model6.add(layers.Dense(5, activation = "softmax"))

model6.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])

model6.summary()

In [ ]:
import datetime
import tensorflow as tf
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback6 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model6_save = ModelCheckpoint('./best_baseline_model6.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop6 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr6 = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

the training and plots

In [ ]:
history= model6.fit(train_ds,
                    validation_data= val_ds,
                    batch_size=BATCH_SIZE,
                    #steps_per_epoch= len(df_train) * 0.8/BATCH_SIZE,
                    #validation_steps= len(df_train) * 0.2/BATCH_SIZE,
                    verbose=1,
                    epochs= 30,
                    callbacks = [tensorboard_callback6, model6_save, early_stop6, reduce_lr6])


# Save the entire model as a SavedModel.
model6.save("drive/MyDrive/cassava-leaf-disease-classification/saved_model/my_model")

In [ ]:
#Plot the Graph
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(loss,'r',linewidth=3.0)
plt.plot(val_loss,'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
  
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(accuracy,'r',linewidth=3.0)
plt.plot(val_accuracy,'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)


In [ ]:
#Evaluate model on test_ds
scores = model6.evaluate(test_ds)


Run prediction on a sample image

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model6.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

Write a function for inference

In [ ]:
import numpy as np
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img.numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    #next two lines fofr debugging
    print('predictions are ', predictions)
    print('predictions[0] are ', predictions[0])
    print('index of max element ', np.argmax(predictions[0]))

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

Now run inference on few sample images

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model6, images[i])
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")